In [82]:
###################################################################################
#                      2. 정형 데이터마이닝 (사용 데이터 : Titanic)                  
###################################################################################

#---------------------------------------------------------------------------------------
# Q1) cabib, embarked변수의 값 중 ""로 처리된 값을 NA로 바꾸고 아래의 데이터 테이블을 보고 
#     문자형, 범주형 변수들을 각각 character, factor형으로 변환하시오.
#     또, 수치형 변수가 NA인 값을 중앙값으로 대체하고, 범주형 변수가 NA인 값을 최빈값으로 대체하고
#     age변수를 아래의 표와 같이 구간화하여 age_1이라는 변수를 생성하고 추가하시오. 
#---------------------------------------------------------------------------------------

import pandas as pd
import numpy as np

# 데이터 수집 => 데이터 전처리 => 모델 훈련 => 모델 성능 평가 => 모델 성능 개선
df = pd.read_csv('./data/모의고사2회/titanic.csv')
print(f"shape={df.shape}")
print(df.head())

# 필드별 NA 개수 확인
print(df.isnull().sum())

# pclass         0
# survived       0
# name           0
# sex            0
# age          263
# sibsp          0
# parch          0
# ticket         0
# fare           1
# cabin       1014
# embarked       2
# dtype: int64

# cabib, embarked변수의 값 중 ""로 처리된 값을 NA로 바꾸고
# cabin값이 비어있는것은 NA로 변환
#df[df['cabin'].isnull()] # 이미 NA로 변환되어 있음
#df[df['embarked'].isnull()] # 이미 NA로 변환되어 있음

# 수치형 변수가 NA인 값을 중앙값으로 대체하고, 범주형 변수가 NA인 값을 최빈값으로 대체
# cabin, embarked의 na값은 중앙값으로 대체(범주형 타입)
df['cabin'] = df['cabin'].fillna(df['cabin'].mode()[0]) # 0번째 원소를 꺼내는 것까지 명시해야함에 주의(최빈값은 여러개 나올수 있으므로 배열로 반환됨)
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

# face는 중앙값으로 대체
df['fare'] = df['fare'].fillna(df['fare'].median())
df['age'] = df['age'].fillna(df['age'].median())

print(df.isnull().sum())
print(df.columns)

def getAgeCode(age):
    if age >= 0 and age < 10:
        return 0
    elif age >= 10 and age < 20:
        return 1
    elif age >= 20 and age < 30:
        return 2
    elif age >= 30 and age < 40:
        return 3
    elif age >= 40 and age < 50:
        return 4
    elif age >= 50 and age < 60:
        return 5
    elif age >= 60 and age < 70:
        return 6
    elif age >= 70 and age < 80:
        return 7
    elif age >= 80 and age < 90:
        return 8

df['age_1'] = df['age'].apply(lambda x : getAgeCode(x))
df['sex_1'] = df['sex'].apply(lambda x : 1 if (x == 'male') else 0)
df['embarked_1'] = df['embarked'].apply(lambda x : 1 if (x == 'S') else (2 if (x == 'C') else 0))

print(df.head())

print(f"shape={df.shape}")
print(df['embarked'].value_counts()) # 범주별 카운트




shape=(1309, 11)
   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

     age  sibsp  parch  ticket      fare    cabin embarked  
0  29.00      0      0   24160  211.3375       B5        S  
1   0.92      1      2  113781  151.5500  C22 C26        S  
2   2.00      1      2  113781  151.5500  C22 C26        S  
3  30.00      1      2  113781  151.5500  C22 C26        S  
4  25.00      1      2  113781  151.5500  C22 C26        S  
pclass         0
survived       0
name           0
sex            0
age          263
sibsp          0
parch          0
ticket         0


In [ ]:
#---------------------------------------------------------------------------------------
# Q2) 전처리가 완료된 titanic 데이터를 train(70%), test(30%) 데이터로 분할하시오.
#    (set.seed(12345)를 실행한 후 데이터를 분할하시오.) 
#    또, train 데이터로 종속변수인 survived(생존 여부)를 독립변수 pclass, sex, sibsp, parch, 
#    fare, embarked로 지정하여 예측하는 분류모델을 3개 이상 생성하고 test 데이터에 대한 
#    예측값을 csv파일로 각각 제출하시오.
#---------------------------------------------------------------------------------------

from sklearn import model_selection
from sklearn.metrics import plot_rot_curve

from sklearn.neighbors import KNeighborsClassifier # KNN

# 선형 모델
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀
from sklearn.svm import LinearSVC # SVM

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df[['pclass', 'sex_1', 'sibsp', 'parch', 'fare', 'embarked_1']]
    , df['survived']
    , train_size = 0.7
    , test_size = 0.3
    , random_state = 0
)

# KNN
# range(a, b) => a 포함, b 제외 사이 정수 반환
for i in range(3,5):
    print(f"n_neighbors=[{i}]")
    model = KNeighborsClassifier(n_neighbors = i)
    model.fit(x_train, y_train)
    print(f"KNN model score={model.score(x_train, y_train):5.3f}")
    print(f"KNN model score={model.score(x_test, y_test):5.3f}")


# 로지스틱 회귀
model = LogisticRegression(max_iter = 5000)
model.fit(x_train, y_train)
print(f"LogisticRegression model score={model.score(x_train, y_train):5.3f}")
print(f"LogisticRegression model score={model.score(x_test, y_test):5.3f}")


# SVM
model = LinearSVC(max_iter = 5000)
model.fit(x_train, y_train)
print(f"SVM model score={model.score(x_train, y_train):5.3f}")
print(f"SVM model score={model.score(x_test, y_test):5.3f}")


# xgboost




n_neighbors=[3]
KNN model score=0.834
KNN model score=0.758
n_neighbors=[4]
KNN model score=0.817
KNN model score=0.746
LogisticRegression model score=0.793
LogisticRegression model score=0.784


In [ ]:
#---------------------------------------------------------------------------------------
# Q3) 생성된 3개의 분류모델에 대해 성과분석을 실시하여 정확도를 비교하여 설명하시오. 
#     또, ROC curve를 그리고 AUC값을 산출하시오.
#---------------------------------------------------------------------------------------